In [1]:
from pathlib import Path

import numpy as np
from lightning.pytorch.callbacks import EarlyStopping, ModelCheckpoint
from matplotlib import pyplot as plt
from PIL import Image
from torch.utils.data import DataLoader
import anomalib
from anomalib.data import PredictDataset, MVTec
from anomalib.engine import Engine
from anomalib.models import Fastflow, EfficientAd, Patchcore, Ganomaly, Draem, Padim
from anomalib.utils.post_processing import superimpose_anomaly_map
from anomalib import TaskType

root = "../../../agricultural_dataset"
cls = "bean"
task = "segmentation"
model_name = "pathcore"
batch_size = 8
num_workers = 8
img_size = 128
test_id = "0000" if cls == "bean" else "000"


if task.upper() == "SEGMENTATION":
    task = TaskType.SEGMENTATION
elif task.upper() == "CLASSIFICATION":
    task = TaskType.CLASSIFICATION
elif task.upper() == "DETECTION":
    task = TaskType.DETECTION
else:
    raise ValueError("Task type not supported")


datamodule = MVTec(
    root= root,
    category= cls,
    train_batch_size= batch_size,
    eval_batch_size= batch_size,
    num_workers= num_workers,
    task=task,
    image_size=(img_size, img_size),
)


model =  Patchcore(
    coreset_sampling_ratio= 0.01
)



model

callbacks = [
    ModelCheckpoint(
        mode="max",
        monitor="pixel_AUROC",
    ),
    EarlyStopping(
        monitor="pixel_AUROC",
        mode="max",
        patience=3,
    ),
]

engine = Engine(
    callbacks=callbacks,
    pixel_metrics="AUROC",
    accelerator="gpu",  # \<"cpu", "gpu", "tpu", "ipu", "hpu", "auto">,
    devices=1,
    logger=False,
)


Could not find wandb. To use this feature, ensure that you have wandb installed.
OpenVINO is not installed. Please install OpenVINO to use OpenVINOInferencer.


In [2]:
# !pip install open_clip_torch

In [3]:
engine.fit(datamodule=datamodule, model=model)

engine.test(datamodule=datamodule, model=model)

Trainer already configured with model summary callbacks: [<class 'lightning.pytorch.callbacks.rich_model_summary.RichModelSummary'>]. Skipping setting a default `ModelSummary` callback.
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
You are using a CUDA device ('NVIDIA GeForce RTX 3090') that has Tensor Cores. To properly utilize them, you should set `torch.set_float32_matmul_precision('medium' | 'high')` which will trade-off precision for performance. For more details, read https://pytorch.org/docs/stable/generated/torch.set_float32_matmul_precision.html#torch.set_float32_matmul_precision
F1Score class exists for backwards compatibility. It will be removed in v1.1. Please use BinaryF1Score from torchmetrics instead
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0,1]
/home/khanhnq/anaconda3/envs/anomalib_env/lib/python3.10/site-packages/lightning/pytorch/core/optimizer.py:182: `LightningModule.configure_optimizers` returned `

┏━━━┳━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━┳━━━━━━━┓
┃   ┃ Name                  ┃ Type                     ┃ Params ┃ Mode  ┃
┡━━━╇━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━╇━━━━━━━┩
│ 0 │ model                 │ PatchcoreModel           │ 24.9 M │ train │
│ 1 │ _transform            │ Compose                  │      0 │ train │
│ 2 │ normalization_metrics │ MetricCollection         │      0 │ train │
│ 3 │ image_threshold       │ F1AdaptiveThreshold      │      0 │ train │
│ 4 │ pixel_threshold       │ F1AdaptiveThreshold      │      0 │ train │
│ 5 │ image_metrics         │ AnomalibMetricCollection │      0 │ train │
│ 6 │ pixel_metrics         │ AnomalibMetricCollection │      0 │ train │
└───┴───────────────────────┴──────────────────────────┴────────┴───────┘

Trainable params: 24.9 M                                                                                           
Non-trainable params: 0                                                                                            
Total params: 24.9 M                                                                                               
Total estimated model params size (MB): 99

Output()

/home/khanhnq/anaconda3/envs/anomalib_env/lib/python3.10/site-packages/lightning/pytorch/loops/optimization/automat
ic.py:132: `training_step` returned `None`. If this was on purpose, ignore this warning...

...

`Trainer.fit` stopped: `max_epochs=1` reached.


F1Score class exists for backwards compatibility. It will be removed in v1.1. Please use BinaryF1Score from torchmetrics instead
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0,1]


Output()

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃        Test metric        ┃       DataLoader 0        ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│        image_AUROC        │    0.9571037292480469     │
│       image_F1Score       │    0.8613953590393066     │
│        pixel_AUROC        │    0.9704117178916931     │
└───────────────────────────┴───────────────────────────┘

[{'pixel_AUROC': 0.9704117178916931,
  'image_AUROC': 0.9571037292480469,
  'image_F1Score': 0.8613953590393066}]

In [4]:
results = engine.test(datamodule=datamodule, model=model)

F1Score class exists for backwards compatibility. It will be removed in v1.1. Please use BinaryF1Score from torchmetrics instead
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0,1]


Output()

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃        Test metric        ┃       DataLoader 0        ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│        image_AUROC        │    0.9571037292480469     │
│       image_F1Score       │    0.8613953590393066     │
│        pixel_AUROC        │    0.9704117178916931     │
└───────────────────────────┴───────────────────────────┘

In [5]:
print(results[0])

{'pixel_AUROC': 0.9704117178916931, 'image_AUROC': 0.9571037292480469, 'image_F1Score': 0.8613953590393066}


In [6]:
print(root + f"/{cls}/test/tear/{test_id}.png")

../../../agricultural_dataset/bean/test/tear/0000.png


In [7]:
import time
start = time.time()
inference_dataset = PredictDataset(path=root + f"/{cls}/test/tear/{test_id}.png")
inference_loader = DataLoader(inference_dataset, batch_size=1, num_workers=num_workers)
end = time.time()

print(f"Time taken to load the image: {end-start}")

Time taken to load the image: 0.0011293888092041016


In [8]:

inference_dataloader = DataLoader(dataset=inference_dataset, num_workers=47)

predictions = engine.predict(model=model, dataloaders=inference_dataloader)[0]

ckpt_path is not provided. Model weights will not be loaded.
F1Score class exists for backwards compatibility. It will be removed in v1.1. Please use BinaryF1Score from torchmetrics instead
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0,1]


Output()

In [9]:

print(
    f'Image Shape: {predictions["image"].shape},\n'
    f'Anomaly Map Shape: {predictions["anomaly_maps"].shape}, \n'
    f'Predicted Mask Shape: {predictions["pred_masks"].shape}',
)

Image Shape: torch.Size([1, 3, 112, 112]),
Anomaly Map Shape: torch.Size([1, 1, 112, 112]), 
Predicted Mask Shape: torch.Size([1, 1, 112, 112])


In [10]:

image_path = predictions["image_path"][0]
image_size = predictions["image"].shape[-2:]
image = np.array(Image.open(image_path).resize(image_size))
print(image.shape)
plt.imshow(image)
plt.axis("off")
plt.show()

(112, 112, 3)


In [11]:
!pwd

/home/khanhnq/Code/Research/computer_vision/APSIPA2024_AgriculturalAD/trainingAD/script


In [12]:
plt.savefig(f"{cls}_image.png")
plt.show()
import cv2
cv2.imwrite(f"{cls}_predict.png", image)

anomaly_map = predictions["anomaly_maps"][0]
anomaly_map = anomaly_map.cpu().numpy().squeeze()
plt.savefig(f"{cls}_anomaly_map.png")
plt.imshow(anomaly_map)
# plt.show()

heat_map = superimpose_anomaly_map(anomaly_map=anomaly_map, image=image, normalize=True)
plt.savefig(f"{cls}_heat_map.png")
plt.imshow(heat_map)
# plt.show()

pred_score = predictions["pred_scores"][0]
pred_labels = predictions["pred_labels"][0]
print(pred_score, pred_labels)

pred_masks = predictions["pred_masks"][0].squeeze().cpu().numpy()
plt.savefig(f"{cls}_pred_masks.png")
plt.imshow(pred_masks)
# plt.show()

tensor(0.6770) tensor(True)
